In [1]:
import pandas as pd
from pandarallel import pandarallel
pandarallel.initialize(nb_workers=8)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
from itertools import islice
import re

def cleanse_text(text):
    return re.sub('[^0-9a-zA-Zㄱ-힗]', '', text)

def window(seq, n=2):
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result
    for elem in it:
        result = result[1:] + (elem,)
        yield result

def preprocess_tags(tags):
    result = list()
    c_tags = [cleanse_text(t) for t in tags]
    for t in c_tags:
        if len(t) < 2:
            result.append(t)
        else:
            for w in window(t):
                result.append(''.join(w))
    return list(set(result + tags))

def preprocess_title(title):
    """
    result = list()
    title = cleanse_text(title)
    if len(title) < 2:
        result.append(title)
    else:
        for w in window(title):
            result.append(''.join(w))
    return result
    """
    return preprocess_tags(title.split(' '))

def preprocess_text(df):
    df['p_title'] = df['plylst_title'].map(lambda x: preprocess_title(x))
    df['p_tags'] = df['tags'].map(lambda x: preprocess_tags(x))

In [3]:
df = pd.read_json('./data/train.json')
test_offset = len(df) - int(len(df) / 10)
train_df, test_df = df[:test_offset], df[test_offset:]
train_df.head(5)
train_df = df

val_df = pd.read_json('./data/val.json')
val_df.head()

preprocess_text(train_df)
preprocess_text(val_df)

In [4]:
val_df[val_df['songs'].map(lambda x: len(x)) == 0]

,tags,id,plylst_title,songs,like_cnt,updt_date,p_title,p_tags
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000,"[테이, 스테, 이블, 리스, 앨리, 앨리스테이블]",[]
8,[스트레스],80810,리듬타면서 빡시게 운동하자!!!(스트레스 날리자):},[],127,2017-02-09 17:33:45.000,"[시게, 운동, 스트, 듬타, 빡시게, 면서, 하자, 리자, 운동하자!!!(스트레스...","[스트, 트레, 레스, 스트레스]"
9,[],142007,기분 좋은 재즈와 함께 만드는 달달한 하루,[],0,2015-06-22 09:11:02.000,"[재즈와, 즈와, 기분, 좋은, 달달, 달한, 드는, 만드는, 만드, 재즈, 달달한...",[]
17,[생각나],2380,다시 생각나는 그 사람,[],6,2016-03-19 23:53:20.000,"[생각나는, 그, 나는, 사람, 생각, 다시, 각나]","[생각, 각나, 생각나]"
20,"[피아노, 이루마, 메로디]",149069,불면증엔 아름다운 뉴에이지 곡들,[],11,2019-12-15 19:15:21.000,"[불면증엔, 아름, 불면, 면증, 에이, 뉴에이지, 이지, 곡들, 뉴에, 아름다운,...","[피아노, 아노, 루마, 메로, 메로디, 이루마, 피아, 이루, 로디]"
...,...,...,...,...,...,...,...,...
22991,[],32537,컨트리 황제 조니 캐시가 선 레코드 시절 발표한 초기 대표작,[],28,2019-06-17 14:22:48.000,"[발표한, 표한, 트리, 대표작, 초기, 코드, 대표, 레코드, 시절, 레코, 캐시...",[]
22992,[사랑],32812,♥옷차림이 가벼워질때 부담없이 듣는음악♥,[],8,2015-05-08 00:09:01.000,"[옷차, ♥옷차림이, 듣는음악♥, 가벼워질때, 부담없이, 림이, 벼워, 담없, 없이...",[사랑]
22996,[],86721,해 저무는 밤,[],5,2016-04-27 15:32:55.000,"[해, 저무, 무는, 밤, 저무는]",[]
23002,"[텐션, 명곡, 학창시절, 텐션업]",56570,숨어듣는명곡! 텐션올릴때듣는 그시절그노래!! (그냥명곡있을수있음주의),[],538,2020-04-18 22:02:07.000,"[있음, 릴때, 노래, 냥명, (그냥명곡있을수있음주의), 수있, 션올, 텐션올릴때듣...","[션업, 학창시절, 텐션업, 창시, 학창, 텐션, 시절, 명곡]"


In [17]:
from khaiii import KhaiiiApi
from itertools import chain

api = KhaiiiApi()
def tokenize(text):
    morphs = list(chain(*[w.morphs for w in api.analyze(text)]))
    return [(m.lex, m.tag) for m in morphs]

In [23]:
[token[0] for token in tokenize("기분전환") if token[1].startswith('N') or  token[1].startswith('V')]

['기분', '전환']

In [15]:
morphs